# Iotics NYX advanced RAG API example

In this example, NYX-subscribed data is used by a [langchain](https://python.langchain.com/v0.2/docs/introduction/) RAG agent to help answer a user's query.
Note that his example leads to almost equivalent behaviour as the [high level one](https://github.com/Iotic-Labs/nyx-sdk/blob/main/examples/high_level/highlevel.ipynb) one but here we illustrate how to put the building blocks together yourself.

📔 Other examples can be found [here](https://github.com/Iotic-Labs/nyx-sdk/blob/main/examples). 📔

## Setup

In [1]:
# Enable auto-reloading (so any updates to installed modules are reflected in subsequent code)
%load_ext autoreload
%autoreload 2

### Prerequisites

Install the Iotics NYX SDK client (with [OpenAI](https://openai.com/) extension). **Note**: You can can also install the `[langchain-cohere]` extra to use [Cohere](https://cohere.com/) instead.

In [ ]:
# Uninstall any previous version first to ensure up-to-date client. (pip -U would instead update ALL packages).
# Note: keep the argument quoted for zsh support.
!pip uninstall -y nyx-client nyx-extras
!pip install nyx_client 'nyx_extras[langchain-openai]'

### Config

Generate configuration .env file via NYX CLI

In [3]:
# NOTE: From a CLI you can also just run: nyx-client init
# (but Jupyter Labs/Notebook don't support stdin via CLI - hence call via Python).
from nyx_client.cli.init_env import init_env

init_env()

Do you want to interactively create '.env'? (y/N):  y

To use Nyx you must agree to our Terms of Service when sharing content


I understand that the data I am uploading will be visible in the Nyx Playground,
including to users outside my organisation.
I confirm that I have the right to share this data.
I confirm that this data does not contain any Personally Identifiable Information
or otherwise sensitive information, and that it does not violate any laws.
I confirm I have read the Nyx Terms of Service and I am content to proceed.
https://www.get-nyx.io/terms

Agree (y/N): y
Enter Nyx URL: https://nyx-playground.iotics.space
Enter Nyx email: some.user@mydomain.com
Enter Nyx password: ··········
Our high level client (NyxLangChain) expects a configured language model, however it can be configured with other language model wrappers provided by langchain providing they support 'tool calling'. Don't forget to set the appropriate API KEY as an environment variable for the language model yo

In [4]:
# Set LLM provider API key (for either Cohere or OpenAI)
from getpass import getpass

llm_api_key = getpass("Enter your backend-specific API key: ")

Enter your backend-specific API key:  ········


## Run

### Initialize

Load the necessary modules.

In [5]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI

from nyx_client import NyxClient, BaseNyxConfig
from nyx_extras.utils import Parser, Utils

### Steps

Set up a generic nyx client

In [6]:
config = BaseNyxConfig.from_env()
client = NyxClient(config=config)

Get all subscribed-to data from nyx and load them into an sq-lite DB (in memory).

📔 Only data in certain formats will be considered (such as CSV). See logging output as to whether some data have been excluded. 📔

In [8]:
# 1. Get all subscribed-to data from Nyx
data = client.my_subscriptions()
print(f"{len(data)} data elements available to ingest")
# 2. Load said data into an SQLite in-memory DB (via SQLAlchemy)
engine = Parser.data_as_db(data)

2 data elements available to ingest


Build an SQL agent using from the [langchain SQL toolkit](https://python.langchain.com/v0.2/docs/integrations/tools/sql_database/) with the just created in-memory database.

In [8]:
# 1. Set up and customize an LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=llm_api_key)
# 2. Wrap the previously created (SQLAlchemy) DB
db = SQLDatabase(engine=engine)
# 3. Create an SQL agent with new llm model and DB
agent_executor = create_sql_agent(llm, db=db, agent_type="tool-calling")

Now we can start to issue queries, enriching them with data from Nyx.

🧠 **Any subscribed-to data will be considered to help answer a query** 🧠

In [9]:
prompt = "What is the land speed record for a car?"
res = agent_executor.invoke({"input": Utils.with_sources(prompt)})
print(res["output"])

I don't know.
